In [1]:
import numpy as np
import pandas as pd
import math
import sys
from ipynb.fs.full.Formulas import *
import random

Bhaichara On Top


# Calculating number of slots for a flow

In [2]:
# Datarate might be wrong, review it later
def calculate_number_of_slots():
    for flow_index in range(1,number_of_flows+1):
        sender = flows[flow_index][2]
        receiver = flows[flow_index][3]
        number_of_slots_for_each_flow[flow_index] = ((throughputs[flow_index] * number_of_time_slots) / datarate_v2v[flow_index])

## Vehicle and UAV class declared here

In [3]:
class Vehicle:
    def __init__(self, x, y, velocity):
        self.x = x
        self.y = y
        self.velocity = velocity
        self.type = 'vehicle'
        
    def calculate_position(self,time):
        current_x=self.x+(self.velocity*time)
        return current_x, self.y
class UAV:
    def __init__(self,x,y,theta,height=height_of_uav,velocity=speed_of_uav,radius=radius_of_uav):
        self.x = x
        self.y = y
        self.height = height
        self.radius = radius
        self.velocity = velocity
        self.theta = theta
        self.type = 'uav'

    def calculate_position(self,time):
        angle = (self.velocity*time)%(2*math.pi)
        current_x=self.x+(self.radius*math.cos(angle))
        current_y=self.y+(self.radius*math.sin(angle))
        return current_x,current_y 


In [4]:
## Relay class models the relaying environment. Environment consists of vehicles and uavs in different positions

In [5]:
class Relay:    
    
    def isbetween(self,middle,x,y):  # Check whether x < middle < y
        left=min(x,y)
        right=max(x,y)
        return middle > left and middle < right
        
    def no_of_obstacles(self,vehicle1,vehicle2,time): # Count no of vehicles between vehicle 1 and vehicle 2 lying in same lane
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        ans=0
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if current_y == vehicle1_y and self.isbetween(current_x,vehicle1_x,vehicle2_x):
                ans+=1
        return ans    

    # Vehicle 1 = sender and Vehicle 2 = receiver
    def determine_relay_type(self,vehicle1,vehicle2,time,flow_index): # Determining type of relaying needed to take place between a flow of sender and receiver and returning candidate relay set if relay is required 
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        if vehicle1_y == vehicle2_y:    #Sender and Receiver on same lane
            obstacles = self.no_of_obstacles(vehicle1,vehicle2,time)
            if obstacles == 0: # If no obstacle, no need to relay
                return 'dont_relay', set()
            elif obstacles == 1: # Only one obstacle, use it to relay
                return 'same_lane_relay', self.candidate_relay_set_single_same_lane(vehicle1,vehicle2,time,flow_index) 
            else: # Multiple obstacles, pick relaying vehicle 
                return 'adjacent_lane_relay', self.candidate_relay_set_multiple_same_lane(vehicle1,vehicle2,time,flow_index)
        elif abs(vehicle1_y - vehicle2_y) == 1: # Sender and Receiver on adjacent lanes
            return 'dont_relay', set()
        else:     # Sender and Receiver are multiple lanes(>1) apart
            return 'middle_lane_relay', self.candidate_relay_set_middle_lane(vehicle1,vehicle2,time,flow_index)
    
    def candidate_relay_set_middle_lane(self,vehicle1,vehicle2,time,flow_index):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_start,vehicle2_y_start=vehicle2.calculate_position(time)

        if vehicle1_y_start > vehicle2_y_start:
            return self.candidate_relay_set_middle_lane(vehicle2,vehicle1,time,flow_index)
        
        vehicle1_x_end,vehicle1_y_end=vehicle1.calculate_position(time+number_of_slots_for_each_flow[flow_index]*slot_duration)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+number_of_slots_for_each_flow[flow_index]*slot_duration)
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+number_of_slots_for_each_flow[flow_index]*slot_duration)
            if self.isbetween(current_y_start,vehicle1_y_start,vehicle2_y_start)==0: 
                continue

            m = current_y_start - vehicle1_y_start
            n = vehicle2_y_start - current_y_start
            D1x , D1y = section_formula(vehicle1_x_start,vehicle1_y_start,vehicle2_x_start,vehicle2_y_start,m,n)
            D2x , D2y = section_formula(vehicle1_x_end,vehicle1_y_end,vehicle2_x_end,vehicle2_y_end,m,n)
            if current_x_end <= D1x or D2x <= current_x_start: # No overlap=No add
                continue
            start_boundary=max(current_x_start,D1x)
            end_boundary = min(current_x_end, D2x)
            epsilon = (end_boundary-start_boundary) / (current_x_end - current_x_start) 
            if epsilon >= 0.5:
                candidate_relays.add(vehicle)        
        # If candidate_relays.length()==0 till now, it means there's no obstacle in middle lanes and no need to relay. so dont add uav and return empty set
        if len(candidate_relays):
            candidate_relays=candidate_relays.union(self.get_uav(vehicle1,vehicle2,time))
        return candidate_relays

    def candidate_relay_set_single_same_lane(self,vehicle1,vehicle2,time,flow_index):
        candidate_relays=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if self.isbetween(current_x,vehicle1_x,vehicle2_x):
                candidate_relays.add(vehicle)
                break
        candidate_relays=candidate_relays.union(self.get_uav(vehicle1,vehicle2,time))
        return candidate_relays

    def candidate_relay_set_multiple_same_lane(self,vehicle1,vehicle2,time,flow_index):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+number_of_slots_for_each_flow[flow_index]*slot_duration)
        D3x=vehicle1_x_start
        D4x=vehicle2_x_end
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+number_of_slots_for_each_flow[flow_index]*slot_duration)
            
            if current_y_start==(vehicle1_y_start-1) or current_y_start==(vehicle1_y_start+1):
                if current_x_end <= D3x or D4x <= current_x_start:
                    continue
                
                start_boundary=max(current_x_start, D3x)
                end_boundary=min(current_x_end, D4x)
                epsilon=(end_boundary-start_boundary)/(current_x_end-current_x_start)
                
                if epsilon>=0.5:
                    candidate_relays.add(vehicle)
                    
        candidate_relays=candidate_relays.union(self.get_uav(vehicle1,vehicle2,time))
        return candidate_relays

    def get_uav(self,vehicle1,vehicle2,time): # Function to return list of possible UAVs available in flow range for relaying
        candidate_uavs=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for uav in candidate_uavs:
            current_x,current_y=uav.calculate_position(time)
            distance_s = ((current_x-vehicle1_x)**2) + ((current_y-vehicle1_y)**2)
            distance_s = math.sqrt(distance_s)
            distance_r = ((current_x-vehicle2_x)**2) + ((current_y-vehicle2_y)**2)
            distance_r = math.sqrt(distance_r)
            if distance_s <= 500 and distance_r <= 500:
                candidate_uavs.add(uav)
        return candidate_uavs
        

In [6]:
# Declare list of vehicles and UAVs objects
list_of_vehicles = [
        Vehicle(10, 1, 75),    
        Vehicle(20, 1, 70),    
        Vehicle(30, 1, 65),    
        Vehicle(40, 1, 60),    
        Vehicle(50, 1, 55),    
        Vehicle(60, 1, 70),    
        Vehicle(70, 1, 72),    
        Vehicle(80, 1, 24),    
        Vehicle(90, 1, 25),   
        Vehicle(100, 1, 50),    
        Vehicle(110, 1, 45),    
        Vehicle(120, 1, 40),    
        Vehicle(130, 1, 35),    
        Vehicle(140, 1, 30),
        Vehicle(10, 11, 75),    
        Vehicle(20, 11, 70),    
        Vehicle(30, 11, 65),    
        Vehicle(40, 11, 60),    
        Vehicle(50, 11, 55),    
        Vehicle(60, 11, 70),    
        Vehicle(70, 11, 72),    
        Vehicle(80, 11, 24),    
        Vehicle(90, 11, 25),   
        Vehicle(100, 11, 50),    
        Vehicle(110, 11, 45),    
        Vehicle(120, 11, 40),    
        Vehicle(130, 11, 35),    
        Vehicle(140, 11, 30),
        Vehicle(10, 2, 75),    
        Vehicle(20, 2, 70),    
        Vehicle(30, 2, 65),    
        Vehicle(40, 2, 60),    
        Vehicle(50, 2, 55),    
        Vehicle(60, 2, 70),    
        Vehicle(70, 2, 72),    
        Vehicle(80, 2, 24),    
        Vehicle(90, 2, 25),   
        Vehicle(100, 2, 50),    
        Vehicle(110, 2, 45),    
        Vehicle(120, 2, 40),    
        Vehicle(130, 2, 35),    
        Vehicle(140, 2, 30),
        Vehicle(10, 12, 75),    
        Vehicle(20, 12, 70),    
        Vehicle(30, 12, 65),    
        Vehicle(40, 12, 60),    
        Vehicle(50, 12, 55),    
        Vehicle(60, 12, 70),    
        Vehicle(70, 12, 72),    
        Vehicle(80, 12, 24),    
        Vehicle(90, 12, 25),   
        Vehicle(100, 12, 50),    
        Vehicle(110, 12, 45),    
        Vehicle(120, 12, 40),    
        Vehicle(130, 12, 35),    
        Vehicle(140, 12, 30),
        Vehicle(10, 21, 75),    
        Vehicle(20, 21, 70),    
        Vehicle(30, 21, 65),    
        Vehicle(40, 21, 60),    
        Vehicle(50, 21, 55),    
        Vehicle(60, 21, 70),    
        Vehicle(70, 21, 72),    
        Vehicle(80, 21, 24),    
        Vehicle(90, 21, 25),   
        Vehicle(100, 21, 50),    
        Vehicle(110, 21, 45),    
        Vehicle(120, 21, 40),    
        Vehicle(130, 21, 35),    
        Vehicle(140, 21, 30),
        Vehicle(10, 31, 75),    
        Vehicle(20, 31, 70),    
        Vehicle(30, 31, 65),    
        Vehicle(40, 31, 60),    
        Vehicle(50, 31, 55),    
        Vehicle(60, 31, 70),    
        Vehicle(70, 31, 72),    
        Vehicle(80, 31, 24),    
        Vehicle(90, 31, 25),   
        Vehicle(100, 31, 50),    
        Vehicle(110, 31, 45),    
        Vehicle(120, 31, 40),    
        Vehicle(130, 31, 35),    
        Vehicle(140, 31, 30)
]

list_of_uavs = [
        UAV(10, 1, math.pi/4),
        UAV(500, 2, math.pi/3),
        UAV(200, 3, math.pi/6)
]

# Create an empty list to store flows
flows = []
flows = [[list_of_vehicles[0],list_of_vehicles[1],0,1]]
# Loop through the list and form flow pairs (sender and receiver pairs)
for i in range(1,len(list_of_vehicles)):
    if len(flows) == (number_of_flows +1):
        break
    # for j in range(i+1, len(list_of_vehicles)):  # Ensure unique pairs
    #     if len(flows) == (number_of_flows + 1):
    #         break
    #     flows.append([list_of_vehicles[i], list_of_vehicles[j],i,j])
    flows.append([list_of_vehicles[0],list_of_vehicles[i],0,i])

# Convert the list of pairs to a NumPy array
flows = np.array(flows)



for i in range(1,len(flows)):
    for j in range(1,len(flows)):
        flow_sender = flows[i][0]
        flow_receiver = flows[j][1]

        sender_x,sender_y = flow_sender.calculate_position(0)
        receiver_x,receiver_y = flow_receiver.calculate_position(0)
        
        distance = euclidean_distance(sender_x, sender_y, receiver_x, receiver_y)
        distance_matrix[i,j] = distance
        

# Testing segment to store standard output in a file (dont run unless required)

In [7]:
# # Open a file in write mode
# with open('output.txt', 'w') as f:
#     # Redirect stdout to the file
#     sys.stdout = f
    
#     # Your print statements
#     temp= Relay()
#     for i in range(5):
#         time = i*slot_duration
#         for j in flows:
#             print("Sender:",j[0].calculate_position(time),"Receiver:",j[1].calculate_position(time))
#             relay_type,candidate_set=temp.determine_relay_type(j[0],j[1],time)
#             for k in candidate_set:
#                 print(k.calculate_position(time))
#             # print(temp.determine_relay_type(j[0],j[1],i*slot_duration))
    
#     # Remember to restore stdout to its default value after writing
#     sys.stdout = sys.__stdout__

# # File is automatically closed after the `with` block

# Creating flows and storing their candidate sets

### This function picks up random candidate out of a particular relay set

In [8]:
def random_relay_candidate(my_set):
    if not my_set:
        return -1
    else:
        return random.choice(list(my_set))

# Calling Functions

In [9]:
# calculate_channel_power_gain_distribution()
# calculate_received_signal_power()
# calculate_sinr_and_datarate_u2v()
# calculate_sinr_and_datarate_v2u()
# calculate_sinr_and_datarate_v2v()
# calculate_number_of_slots()
# flows_candidate_relay_set_pairs={}
# RelayObj = Relay()

In [10]:
calculate_channel_power_gain_distribution()
calculate_received_signal_power()
calculate_sinr_and_datarate_u2v()
calculate_sinr_and_datarate_v2u()
calculate_sinr_and_datarate_v2v()
calculate_number_of_slots()
flows_candidate_relay_set_pairs={}
RelayObj = Relay()

/run/media/prateek/Documents/joint_optimization_relay_selection/Formulas.ipynb:172: RuntimeWarning: divide by zero encountered in scalar power
  "execution_count": 7,
/run/media/prateek/Documents/joint_optimization_relay_selection/Formulas.ipynb:172: RuntimeWarning: invalid value encountered in scalar multiply
  "execution_count": 7,
/run/media/prateek/Documents/joint_optimization_relay_selection/Formulas.ipynb:186: RuntimeWarning: divide by zero encountered in scalar power
  "execution_count": 8,
/run/media/prateek/Documents/joint_optimization_relay_selection/Formulas.ipynb:186: RuntimeWarning: invalid value encountered in scalar multiply
  "execution_count": 8,
/tmp/ipykernel_17308/3936472145.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  number_of_slots_for_each_flow[flow_index] = ((throughputs[flow_index] * number_of_time_slots) / datarate_v2v[flow_index])


### Mapping flows to their respective (relay type,candidate set)

In [11]:

# for current_time_slot in range(number_of_time_slots):
#     time = current_time_slot * slot_duration
#     for flow in flows:
#         flow_with_time = [flow[0].calculate_position(time), flow[1].calculate_position(time), time]
#         flow_with_time = str(flow_with_time)
#         relay_type,candidate_set = RelayObj.determine_relay_type(flow[0],flow[1],time)
#         flows_candidate_relay_set_pairs[flow_with_time] = [relay_type,candidate_set,random_relay_candidate(candidate_set)]
for current_time_slot in range(number_of_time_slots):
    time = current_time_slot * slot_duration
    for flow_index in range(1,len(flows)):
        if (current_time_slot+number_of_slots_for_each_flow[flow_index])>=number_of_time_slots:
            break
        flow_with_time = [flows[flow_index-1][0].calculate_position(time), flows[flow_index-1][1].calculate_position(time), time]
        flow_with_time = str(flow_with_time)
        relay_type,candidate_set = RelayObj.determine_relay_type(flows[flow_index-1][0],flows[flow_index-1][1],time,flow_index)
        flows_candidate_relay_set_pairs[flow_with_time] = [relay_type,candidate_set,random_relay_candidate(candidate_set)]

In [12]:
print(number_of_slots_for_each_flow)

[ 0.00000000e+00 -1.69803240e+12 -1.85740425e+13 -6.27201528e+13
 -8.28032655e+13 -5.51923101e+14 -1.23872669e+15 -2.06420889e+15
 -1.20412014e+15 -3.59788212e+14 -4.66197219e+14 -1.09483906e+15
 -4.21041428e+14 -1.16458345e+15 -2.05007090e+12 -9.78222589e+12
 -2.54945739e+12 -2.90291367e+13 -2.44760487e+13 -2.12827409e+14
 -1.19279738e+14 -8.30937644e+14 -1.34775801e+14 -3.22118216e+14
 -4.42826181e+14 -1.07985312e+15 -2.54949849e+15 -7.63957943e+15
 -3.75912178e+10 -3.33150966e+12 -3.11570496e+13 -5.32497004e+12
 -5.73217869e+13 -1.59898891e+14 -2.59266587e+14 -2.89065916e+14
 -1.15150019e+15 -1.95899392e+15 -1.69733970e+15             inf
 -1.96397240e+15 -1.08297335e+15 -2.14387211e+13 -7.98484970e+12
 -6.59092247e+13 -1.92115372e+14 -4.50699659e+14 -3.64635336e+14
 -1.48989749e+14 -3.66794178e+15 -5.34794929e+14 -3.09960297e+14
 -5.57237093e+14 -2.86822118e+15 -3.18730775e+14 -1.96593700e+15
 -2.32348048e+14 -1.54776311e+13 -3.50508516e+13 -4.58171729e+13
 -5.84265891e+14 -6.33355

In [13]:
print(received_signal_power_matrix)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.65009787e-08 1.16671982e-08 ... 1.14429864e-10
  4.80551369e-11 1.19586509e-10]
 [0.00000000e+00 4.24576444e-08 1.13290724e-08 ... 1.04410550e-10
  1.57352830e-10 2.29976433e-10]
 ...
 [0.00000000e+00 1.26380052e-07 1.15199278e-08 ... 8.69250668e-11
  1.08155239e-10 4.18577223e-11]
 [0.00000000e+00 2.83606354e-08 1.01031920e-08 ... 5.78435103e-10
  2.49905025e-10 7.83359704e-11]
 [0.00000000e+00 1.01635224e-08 6.93524537e-09 ... 8.37598321e-10
  6.53008879e-12 8.71853091e-11]]


In [14]:
print(channel_power_gain_matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.80767973 1.46041484 ... 0.54029087 0.29476616 0.93196582]
 [0.         0.93948655 1.41809072 ... 0.49298378 0.96518901 1.79226048]
 ...
 [0.         2.79648956 1.44198061 ... 0.4104245  0.66341513 0.32620708]
 [0.         0.62755332 1.26464395 ... 2.73113325 1.53289638 0.61049066]
 [0.         0.22489455 0.86810347 ... 3.95479563 0.04005501 0.67945564]]


In [15]:
channel_power_gain_matrix.shape

(81, 81)

In [16]:
distance_matrix[1]

array([  0.        ,  10.        ,  20.        ,  30.        ,
        40.        ,  50.        ,  60.        ,  70.        ,
        80.        ,  90.        , 100.        , 110.        ,
       120.        , 130.        ,  10.        ,  14.14213562,
        22.36067977,  31.6227766 ,  41.23105626,  50.99019514,
        60.8276253 ,  70.71067812,  80.62257748,  90.55385138,
       100.49875621, 110.45361017, 120.41594579, 130.3840481 ,
         1.        ,  10.04987562,  20.02498439,  30.01666204,
        40.01249805,  50.009999  ,  60.00833275,  70.00714249,
        80.00624976,  90.00555538, 100.00499988, 110.00454536,
       120.00416659, 130.0038461 ,  11.        ,  14.86606875,
        22.82542442,  31.95309062,  41.48493703,  51.19570294,
        61.        ,  70.85901495,  80.75270893,  90.66973034,
       100.60318086, 110.54863183, 120.50311199, 130.46455457,
        20.        ,  22.36067977,  28.28427125,  36.05551275,
        44.72135955,  53.85164807,  63.2455532 ,  72.80

In [17]:
distance_matrix.shape

(81, 81)

In [18]:
type(distance_matrix)

numpy.ndarray

In [19]:
import pandas as pd
# Define the file path where you want to save the CSV file
file_path = "data.csv"

# Save the NumPy array as a CSV file
np.savetxt(file_path, distance_matrix, delimiter=",")

In [20]:
len(flows)

81

In [21]:
len(list_of_vehicles)

84

In [22]:
datarate_v2v

array([ 0.00000000e+00, -3.14448346e-10, -9.76404398e-11, -2.63962344e-11,
       -1.81954625e-11, -3.07528944e-12, -1.53764472e-12, -7.68822361e-13,
       -1.53764472e-12, -3.58783768e-12, -3.33156356e-12, -1.28137060e-12,
       -1.28137060e-12, -1.02509648e-12, -5.32793896e-10, -1.26599415e-10,
       -1.04047293e-10, -2.25521226e-11, -1.30699801e-11, -9.22586833e-12,
       -4.86920829e-12, -1.28137060e-12, -6.91940125e-12, -2.05019296e-12,
       -1.28137060e-12, -1.53764472e-12, -7.68822361e-13, -2.56274120e-13,
       -4.42552090e-08, -1.43257233e-10, -4.84358087e-11, -1.33006268e-10,
       -2.40897673e-11, -7.17567537e-12, -7.17567537e-12, -4.10038592e-12,
       -5.12548241e-13, -7.68822361e-13, -7.68822361e-13,  0.00000000e+00,
       -7.68822361e-13, -7.68822361e-13, -8.22639926e-11, -1.92718138e-10,
       -7.68822361e-12, -9.73841657e-12, -3.33156356e-12, -5.38175653e-12,
       -1.79391884e-12, -5.12548241e-13, -2.05019296e-12, -2.56274120e-12,
       -1.53764472e-12, -

In [23]:
len(datarate_v2v)

81

In [24]:
sinr_v2v

array([ 0.00000000e+00, -1.36197682e-13, -4.22726584e-14, -1.14670588e-14,
       -7.85791113e-15, -1.31177942e-15, -6.26522346e-16, -3.86922584e-16,
       -7.09219542e-16, -1.59999726e-15, -1.41523449e-15, -5.51037986e-16,
       -5.12335049e-16, -4.09587275e-16, -2.30775833e-13, -5.48155102e-14,
       -4.50367066e-14, -9.82268426e-15, -5.64154350e-15, -4.00086315e-15,
       -2.07471372e-15, -5.69353452e-16, -2.99560052e-15, -9.42726728e-16,
       -5.32867413e-16, -6.24057811e-16, -3.79186578e-16, -6.02525970e-17,
       -1.91721614e-11, -6.20867538e-14, -2.09767887e-14, -5.76042841e-14,
       -1.04391550e-14, -3.08995071e-15, -3.10473425e-15, -1.75713374e-15,
       -2.11570005e-16, -3.27250515e-16, -3.79465681e-16, -4.56214116e-17,
       -2.97827629e-16, -3.18753129e-16, -3.55933027e-14, -8.34333885e-14,
       -3.36797403e-15, -4.18902865e-15, -1.40578548e-15, -2.29387453e-15,
       -7.23607798e-16, -2.13542346e-16, -8.94019933e-16, -1.11747183e-15,
       -6.71709140e-16, -